Empecemos estableciendo la estructura porque si no esta muy duro como empezar a escribir este trabajo:

La escritura de este proyecto esta compleja, porque sinceramente, no se que voy a presentar aun.

In [3]:
# Empecemos con los imports

from ase.build import molecule
from ase.io import read, write
from ase.visualize import view
from ase.optimize import BFGS, QuasiNewton
from ase.vibrations import Vibrations

from mace.calculators import mace_mp , MACECalculator
import numpy as np
import matplotlib.pyplot as plt


# Modelo MACE como una alternativa al DFT en la construcción de potenciales interatomicos

Daniel José Duque $^{1}$ , Tomas Uberney Goméz $^{1}$ 

$ ^{1}\textit{Instituto de Física, Universidad de Antioquia U de A, calle 70 No. 52-21, Medellín, Colombia} $

### Introducción

El proyecto busca abordar el problema del cálculo eficiente y preciso de las energías y fuerzas interatómicas en sistemas moleculares y materiales, una tarea fundamental en química computacional y física del estado sólido. Tradicionalmente, este tipo de cálculos se realiza mediante métodos ab initio ( cálculos de primeros principios ), como la Teoría del Funcional de la Densidad (DFT), los cuales, aunque altamente precisos, resultan extremadamente costosos en términos computacionales cuando se aplican a sistemas grandes o dinámicos. En respuesta a esta limitación, el modelo MACE (Multiplicative Atomic Cluster Expansion) propone una solución basada en aprendizaje profundo (Deep Learning), que permite aproximar la precisión del DFT con un costo computacional mucho menor.

El desarrollo de modelos como MACE representa un avance interdisciplinario clave entre la física, la química computacional y el aprendizaje estadístico. En un contexto científico y tecnológico, este tipo de modelos democratizan el acceso a simulaciones cuánticas de alta precisión, permitiendo estudiar materiales complejos, procesos catalíticos, o interacciones biológicas que antes requerían recursos de supercomputación. En particular este proyecto nace del interés de estudiar las propiedades  del cristal de beta-hematina, el cual es un dímero de ferriprotoporfirina IX (Fe(III)PPIX), cada monómero de hematina tiene la forma molecular aproximada de C34​H32​FeN4​O4, así el dímero se representa mediante C68​H62​Fe2​N8​O8​​ es decir un total de 148 átomos por dímero. Pagola et al. reportan un modelo donde por celda unitaria del cristal existen 2 dímeros (es decir 296 átomos por celda unitaria), ahora bien si se quisiese simular un cristal crecido se requiere de múltiplos enteros de estas celdas unitarias, elevando fuertemente el coste computacional de simular estas interacciones interatómicas por primeros principios. Al emplear MACE estamos refiriéndo nos a cálculos del orden de ~O(N), procesos que no demandan mucho coste computacional.

Así pues la comprensión de MACE como herramienta práctica para el desarrollo de calculos y simulaciones se ve como una hábilidad útil en el desarrollo de nuestra carrera profesional, de modo que se plantean una serie de metas en un primer acercamiento a esta MLIP:

- Aprender a usar MACE incorporado en ASE y así poder determinar las energías de nuestros sistemas atómicos y cómo deducir propiedades deseadas a partir de esta información.
- Entender el proceso del fine-tuning y cómo llevarlo a cabo a partir de datos experimentales o de laboratorio.
- Emplear MACE para el cálculo de propiedades elásticas del dímero de Beta-Hematina y su comparación cualitativa con un espectro IR de una muestra experimental. (Este objetivo depende del tiempo disponible)
- Entender el funcionamiento de MACE como GNN y MPNN, desde su funcionamiento como algoritmo, hasta su fase de entrenamiento como “learnable model” 


## Metodologia

### MACE como MLIP

MACE es una equivariance MPNN, es decir una red neuronal equivariante de intercambio de mensajes, este tipo de redes son redes basadas en gráfos, donde la estructura de los datos se representa mediante nodos y sus aristas correspondientes 

![texto alternativo](./Imagenes/algoritmo_MPNN.jpg)

En este tipo de redes los features se asocian a los nodos $\sigma$ mediante tuplas, dichas tuplas cuentan con varias entradas posibles, primero, dentro de ellas se alamacenan 1. Los números atomicos de cada nodo, 2. sus coordenadas espaciales respecto a un sistema de referencia y 3. lo que se denomina como vector de features $\vec{h}$, este último aunque se denomine vector posee un caracter tensorial, pues primero tiene diferentes canales de información, así como se representa mediante bases de mayor rango (Esto con el fin de representar el acoplamiento de los estados cuanticos). Además tanto al nodo como al mensaje se les asocia una iteracción (correspondiente a la cantida de mensajes construidos y compartidos hasta dicho momento).

Una MPNN tiene un funcionamiento básico, se inicializa y posteriormente empieza a construir interacciónes entre sus elementos internos, estas interacciónes se pueden clasificar dependiendo del número de cuerpos que interactuen simultaneamente, estas son conocidas como mensajes. Tras que cada nodo construya si mensaje, procede a actualizar sus features mediante funciones de aprendizaje. 

Una vez recorrido un número de ciclos preestablecidos (Los autores comentan que 2 o 3 son más que suficientes), se procede a construir un mensaje de salida, este es el target de nuestra red (correspondiendo en este caso a la energía del estado elemental del sistema) y es un invariante, este target se obtiene mediante unas readout functions, que son nuevamente funciones de aprendizaje.

Con el fin de aproximar lo mejor posible la realidad se ha establecido que estas redes deven ser equivariantes, es decir sus features deben transformar de una manera adecuada frente a la acción de un grupo, en partícular es de especial interes el que se transformen bajo $O(3)$ 

![texto alternativo](./Imagenes/equivarianza.jpg)

Esta equivarianza se representa bajo las matrices $D$ generalizaciones del grupo sobre espacios vectoriales de mayor dimensión.

Así, MACE integra las MPNNs con la equivarianza a través de un ingenioso truco y es la construcción del mensaje pensandolo como un producto tensorial el cual da cuenta de la superposición del estado de unas bases que respetan precisamente la equivarianza sobre este grupo $O(3)$.

![texto alternativo](./Imagenes/MACE_MPNN.jpg)

Precisamente este producto tensorial es clave en la mejora sustancial del tiempo de computo de MACE, pues implica que no debemos jerarquizar las interacciones, de hecho permite precisamente autointeracciones de muchos ordenes, asi el computo se reduce realmente a un pequeño número de productos matriciales.

Dentro de la construcción del mensaje se emplean unas bases radiales derivadas de las funciones de Bessel y ademas incorporamos los Armonicos esfericos, acoplados por los coeficiente de Clebsh-Gordan generalizado. A lo largo de la construcción del mensaje se emplean diversos coeficiente $W$ que son funciones de aprendizaje.

La salida del sistema es un invariante, pues representa la energia, para esto se emplean unas read-out functions que unicamente emplean el componente $L=0$ que corresponde precisamente a los invaraintes del grupo (escalares). La salida se presenta como una expansión donde cada termino esta asociado a uno de los pasos (iteraciones de la red). Para la última iteración el readout se obtiene mediante un multilayer-perceptron (fullyconnected neural network) y para los pasos  intermedios se se emplean unos pesos aprendidos.

### ¿Cómo se emplea MACE?

Desde la perspectiva del usuario final MACE es un calculador de las energías del estado base de un sistema molecular, como calculador se puede asociar a diferentes softwares de simulaciónes atomicas, principalmente esta pensado para incorporarse en la libreria de Python ASE (Atomic Similation Enviroment), pero tambien se puede emplear en LAMMPS o en OpenMM (Para su implementación consultar en https://mace-docs.readthedocs.io/en/latest/guide/openmm.html), de modo que trabajar con MACE se puede dividir unicamente en 2 procesos,  **Entrenamiento** y **MACECalculator**. 

El primero se divide en 2 categorias, un entrenamiento desde cero, donde los parametros del modelo se inicializan aleatoriamente y se entrena a partir de un dataset de preferencia o tambien se encuentra lo que comunmente se denomina co 